# Fonction de base de theano

Pour commencer voyons les quelques fonctions "basiques" de theano dont nous auront besoin pour la suite de ces tutoriels

## Déclarer des variables et executer des fonctions

Tout d'abord nous avons besoin d'importer différentes librairie python ainsi que theano:

In [1]:
from theano import *
import theano.tensor as T
from theano import function
import numpy

### Les variables

Nous allons maintenant voir les différents types de variables que nous pouvons utiliser avec theano.

#### Les variables dites "statiques"

Nous allons commencer par les variables "classiques", elles peuvent prendre plusieurs type donnés ci après:
* byte: bscalar, bvector, bmatrix, brow, bcol, btensor3, btensor4, btensor5, btensor6, btensor7
* 16-bit integers: wscalar, wvector, wmatrix, wrow, wcol, wtensor3, wtensor4, wtensor5, wtensor6, wtensor7
* 32-bit integers: iscalar, ivector, imatrix, irow, icol, itensor3, itensor4, itensor5, itensor6, itensor7
* 64-bit integers: lscalar, lvector, lmatrix, lrow, lcol, ltensor3, ltensor4, ltensor5, ltensor6, ltensor7
* float: fscalar, fvector, fmatrix, frow, fcol, ftensor3, ftensor4, ftensor5, ftensor6, ftensor7
* double: dscalar, dvector, dmatrix, drow, dcol, dtensor3, dtensor4, dtensor5, dtensor6, dtensor7
* complex: cscalar, cvector, cmatrix, crow, ccol, ctensor3, ctensor4, ctensor5, ctensor6, ctensor7

Pour déclarer une variable il suffit de lui indiquer son nom, son type et une string pour la décrire. Toutes les types de variables sont contenus dans theano.tensor. Voyons un example où nous déclarons x un dscalar:

In [2]:
x = T.dscalar('x')

Nous pouvons essayer avec plusieurs type de variables, essayez de déclarer une matrice a de double et un vecteur b de float:

In [3]:
# enlevez les dies avant de compiler
# a = ...
# b = ...

#### Les variables partagées

Nous pouvons aussi déclarer des variables partagées dont la valeur pourra être modifiée et sauvegarder lors des appels de fonctions ( que nous verrons plus loin ). Pour cela il suffit d'indiquer la valeur de départ de la variable et son nom:

In [4]:
state = theano.shared(0)

La fonction shared de theano peut prendre tous les types de variables, example avec un tableau de valeurs:

In [32]:
x = theano.shared(  value=numpy.zeros((0,10),dtype=theano.config.floatX),  name='x',  borrow=True)

In [34]:
print(x.get_value())

[]


### Les fonctions avec theano

Afin d'executer des opérations sous theano il faut déclarer des fonction que l'on appelera, pour cela il faut au préalable définir les operations à effectuer ainsi que déclarer les variables de ces opérations:

In [6]:
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y

Puis on définit la fonction, pour cela nous devons indiquer les variables qui doivent être passées en arguments lors de l'appel ainsi que l'opération concernée:

In [7]:
f = function([x, y], z)

On peut maintenant executer notre fonction:

In [8]:
f(2, 3)

array(5.0)

On peut aussi effectuer des operations entre des variables de taille differentes, exemple:

In [9]:
x = T.dscalar('x')
y = T.dmatrix('y')
z = x + y
f = function([x, y], z)
f(2, [[3,2],[2,3]])

array([[ 5.,  4.],
       [ 4.,  5.]])

Il est aussi possible d'executer plusieur opérations en même temps grâce à un appel de fonction:

In [10]:
a, b = T.dmatrices('a', 'b')
y = T.dscalar('y')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
z = y + diff_squared
f = theano.function([a, b, y], [diff, abs_diff, diff_squared, z])
f([[1, 1], [1, 1]], [[0, 1], [2, 3]], 5)

[array([[ 1.,  0.],
        [-1., -2.]]), array([[ 1.,  0.],
        [ 1.,  2.]]), array([[ 1.,  0.],
        [ 1.,  4.]]), array([[ 6.,  5.],
        [ 6.,  9.]])]

#### Les fonctions pour les variables partagées

Comme vu plus haut les variables partagées sauvegardent leurs données à chaque appel de fonction, on peut voir ces variables comme des compteurs sur lequel nous allons appliquer des opérations, l'exemple ci dessous montre une fonction comprenant une variable partagée:

In [11]:
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

On peut observer la valeur de notre variable à n'importe quel moment grâce à la fonction state.get_value() :

In [12]:
print(state.get_value())

0


In [13]:
accumulator(1)

array(0)

In [14]:
print(state.get_value())

1


In [15]:
accumulator(200)

array(1)

In [16]:
print(state.get_value())

201


#### Initialiser un argument dans une fonction

On peut, lors de la déclaration d'une fonction, initialiser un de ses arguments avec une valeur par défaut, cela nous permettra de pouvoir appeler cette fonction avec ou sans cet argument. Les arguments initialisés par une valeur par défaut doivent se trouver en fin de déclaration de la fonction.

In [17]:
x, y = T.dscalars('x', 'y')
z = x + y
f = function([x, In(y, value=1)], z)

In [18]:
f(33)

array(34.0)

In [19]:
f(33,2)

array(35.0)

#### Les fonctions random

Pour se servir des fonctions random de la librairie theano il faut les importer:

In [20]:
from theano.tensor.shared_randomstreams import RandomStreams

On peut maintenant utiliser les différentes fonctions random de la librairie, pour commencer nous devons instancier un seed pour générer les nombres aléatoirement:

In [21]:
srng = RandomStreams(seed=234)

Puis nous pouvons créer les oprerations qui générerons les nombres aléatoires, pour cela plusieurs methodes nous sont proposées par la librairie theano, ici nous utilisons les methodes uniform et normal:

In [26]:
rv_u = srng.uniform((2,2)) #nous indiquons dans la fonction les dimensions que prend notre variable aléatoire, ici une matrice 2,2
rv_n = srng.normal((2,2))

Afin il nous suffit de déclarer des fonctions executant ces operations et de les appeler:

In [24]:
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)
f()

array([[ 0.31971415,  0.47584377],
       [ 0.24129163,  0.42046081]])

In [25]:
g()

array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]])

## Cas pratique

Maintenant que nous avons vu comment déclarer des variables ainsi que les différentes notions sur les fonctions avec theano nous allons essayer de programmer un modèle basique de réseau de neurone avec theano, pour cet exemple nous nous baserons sur le modèle de la "Logistic Regression":

Pour commencer importons les librairies qu'il nous manque pour cet exemple:

In [27]:
from __future__ import print_function

Voici le code du programme, certaine partie sont a completer par vous même:

In [ ]:
rng = numpy.random

N = 400                                   # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = #declarez x une matrice de double
y = #declarez y un vecteur de double

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = #déclarez w les poids du graphe en tant que variable partagée initialisée de façon aléatoire en fonction de feats (rng.randn(feats))

# initialize the bias term
b = #déclarez w les biais du graphe en tant que variable partagée initialisée à 0

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = #déclarez une fonction avec les paramètres suivants:
                              #entrées : x,y
                              #sorties : prediction, xent
                              #updates : (w, w - 0.1 * gw), (b, b - 0.1 * gb)
predict = #déclarez une fonction avec les paramètres suivants:
                              #entrée : x
                              #sortie : prediction

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))